In [1]:
import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, BatchNormalization


In [2]:
(x, y), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [3]:
x.shape

(50000, 32, 32, 3)

In [4]:
x_test.shape

(10000, 32, 32, 3)

In [5]:
# tensorflow Dataset class.
train_dataset = tf.data.Dataset.from_tensor_slices((x, y))

In [6]:
HEIGHT = 32
WIDTH = 32
NUM_CHANNELS = 3
NUM_CLASSES = 10

In [7]:
def augmentation(x, y):
    x = tf.image.resize_with_crop_or_pad(
        x, HEIGHT + 8, WIDTH + 8)
    x = tf.image.random_crop(x, [HEIGHT, WIDTH, NUM_CHANNELS])
#     x = tf.image.random_flip_left_right(x)
    return x, y

In [8]:
train_dataset = train_dataset.map(augmentation)

In [9]:
train_dataset = (train_dataset
                 .map(augmentation)
                 .shuffle(buffer_size=50000))

In [10]:
def normalize(x, y):
  x = tf.cast(x, tf.float32)
  x /= 255.0  # normalize to [0,1] range
  return x, y

In [11]:
train_dataset = (train_dataset
                 .map(augmentation)
                 .shuffle(buffer_size=50000)
                 .map(normalize))

In [12]:
train_dataset = (train_dataset.map(augmentation)
                 .map(normalize)
                 .shuffle(50000)
                 .batch(128, drop_remainder=True))

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = (test_dataset.map(normalize).batch(128, drop_remainder=True))

In [14]:
NUM_GPUS = 1

BS_PER_GPU = 128
NUM_EPOCHS = 60
NUM_TRAIN_SAMPLES = 50000

BASE_LEARNING_RATE = 0.1
LR_SCHEDULE = [(0.1, 30), (0.01, 45)]



def schedule(epoch):
  initial_learning_rate = BASE_LEARNING_RATE * BS_PER_GPU / 128
  learning_rate = initial_learning_rate
  for mult, start_epoch in LR_SCHEDULE:
    if epoch >= start_epoch:
      learning_rate = initial_learning_rate * mult
    else:
      break
  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate

In [15]:
i = Input(shape=(HEIGHT, WIDTH, NUM_CHANNELS))
x = Conv2D(32, (3, 3), activation='relu')(i)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
 
# last hidden layer i.e.. output layer
x = Dense(NUM_CLASSES)(x)
 
model = Model(i, x)

In [16]:
# i = Input(shape=(HEIGHT, WIDTH, NUM_CHANNELS))
# x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
# x = BatchNormalization()(x)
# x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D((2, 2))(x)
 
# x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D((2, 2))(x)
 
# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D((2, 2))(x)
 
# x = Flatten()(x)
# x = Dropout(0.2)(x)
 
# # Hidden layer
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.2)(x)
 
# # last hidden layer i.e.. output layer
# x = Dense(NUM_CLASSES, activation='softmax')(x)
 
# model = Model(i, x)

In [17]:
model.compile(
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam'),
          metrics=['accuracy'])

In [18]:
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()
tensorboard_callback = TensorBoard(
  log_dir=log_dir,
  update_freq='batch',
  histogram_freq=1)

lr_schedule_callback = LearningRateScheduler(schedule)

model.fit(train_dataset,
          epochs=NUM_EPOCHS,
          validation_data=test_dataset,
          validation_freq=1,
          callbacks=[tensorboard_callback, lr_schedule_callback])
model.evaluate(test_dataset)

model.save('cifar_model.h5')

Epoch 1/60
390/390 [==============================] - 37s 56ms/step - loss: 2.6124 - accuracy: 0.0983 - val_loss: 57509.1406 - val_accuracy: 0.1001
Epoch 2/60
190/390 [=============>................] - ETA: 12s - loss: 2.3079 - accuracy: 0.0977

KeyboardInterrupt: 

In [ ]:
new_model = keras.models.load_model('model.h5')
 
new_model.evaluate(test_dataset)